# Fills
There are situations when the range of the actual boundary computation of an operator does not coincide with the range of a kernel function. In some of these situations, it is important to take some other form of action than the default action. The default action is to perform no computation when the thread count exceeds the data array size of a kernel. This default action does not always lead to correct computations. For instance, issues in the form of erroneous computational results arise when operators are nested because that typically causes the size of boundary data region to grow. The problem with that is that the boundary data region of the nesting now contains a mixture of both boundary and interior stencils. The fill module provides functions and classes for expanding the boundary data arrays by filling them with zeros, or interior stencils, as well as updating the index bounds accordingly. 





For example, suppose that an operator is defined as follows

In [1]:
from openfd.alpha import Operator, array, GridFunction, Expression, index
import numpy as np

class Derivative(Operator):    
    # Called during object initialization (avoids the need to having to overload the init method and calling super)
    def initialize(self):
        # data key refers to region id     
        self._data[0] = array.CArray('dl', data = np.array([[-1.0, 1.0]]))
        self._data[1] = array.CArray('di', data = np.array([-0.5, 0.5]))
        self._data[2] = array.CArray('dr', data = np.array([[-1.0, 1.0]]))
    
    def __getitem__(self, indices):
        left = self.data[0]
        interior = self.data[1]
        right = self.data[2]
        # Handle left boundary region
        if indices.id == 0:
            return sum([self.args[i]*left[indices, i] for i in range(left.shape[1])])
        # Handle interior region
        elif indices.id == 1:
            return self.args[indices-1]*interior[0] + self.args[indices+1]*interior[1]
        # Handle right boundary region
        elif indices.id == 2:
            return sum([self.args[-(i+1)]*right[indices, i] for i in range(right.shape[1])])
        else:
            raise NotImplementedError('Unable to handle region: %d', indices.id)


        
#place-holder class that will be implemented if approved. Will also include bounds as discussed in issue #13
# These three classes will be one class. The region id will be an attribute.
from sympy import Symbol
class LeftBoundaryIndex(Symbol):
    @property
    def id(self):
        return 0
    
class RightBoundaryIndex(Symbol):
    @property
    def id(self):
        return 2
    
class InteriorIndex(Symbol):
    @property
    def id(self):
        return 1
      

Then to use this operator, we need

In [2]:
D = Derivative('D')
u = GridFunction('u', shape=(10,) )
i_left = LeftBoundaryIndex('il')
i_int = InteriorIndex('ii')
i_right = RightBoundaryIndex('ir')
expr = Expression(D*u)  

Left boundary evaluation gives

In [3]:
expr[i_left]

u[0]*dl[il][0] + u[1]*dl[il][1]

Interior evaluation gives

In [4]:
expr[i_int]

u[ii - 1]*di[0] + u[ii + 1]*di[1]

Right boundary evaluation gives

In [5]:
expr[i_right]

u[-2]*dr[ir][1] + u[-1]*dr[ir][0]

Now, if we use the kernel function generator to produce code for the computation of the left boundary using say 4 threads, then no computation will take place for three of those threads because the data array `left` only contains one row of coefficients (corresponding to one grid point, furthest to the left). Sometimes this default "no computation" action is the desired action to take and other times it is not. In fact, there are certain scenarios where it may lead to the wrong computational result! 

What can we do instead? One option is to perform the interior computation for these stencils as well. For that to work. Two tasks need to be sorted out:
1. The `left` array needs to be filled with three more rows holding the interior stencil (banding)
2. The interior region bounds needs to be updated to adjust for this change (otherwise this computation runs the risk of being performed twice for each of the three modified points).

## Banded fill-in
A banded fill-in is when the extra space that opens up in a boundary region is filled with repeated interior stencils that give rise to a banded structure. For example, filling the array `left` with the three nearest interior points would result in the following data array


$$\begin{align}
A = 
\begin{bmatrix}
-1 & 1 & 0 & 0 & 0 \\
 -0.5 & 0 & 0.5 & 0 & 0 \\
 0 & -0.5 & 0 & 0.5 & 0 \\
 0 & 0 & -0.5 & 0 & 0.5
\end{bmatrix}
\end{align}
$$

If done manually, the code could look something like this

In [6]:
banded_left = [[-1.0, 1.0, 0.0, 0.0, 0.0], 
               [ -0.5, 0.0, 0.5, 0.0, 0.0],
               [0.0, -0.5, 0.0, 0.5, 0.0],
               [ 0.0, 0.0, -0.5, 0.0, 0.5]]
left = array.CArray('left', np.array(banded_left))

The same result can be accomplished using 

In [7]:
from openfd.alpha import fills
banded_left = fills.banded(np.array([-0.5, 0, 0.5]), num_repeats=3,  block=np.array([-1.0, 1.0]), reverse=False)

ImportError: cannot import name fills

See the API documentation [fills.banded](fills.banded) (TODO: fix this reference) for an explanation of what the arguments are.

## Zero fill-in
An alternative to banded fill-in is zero fill-in. This fills the data array with zeros after the boundary block. This feature could be useful for overcoming issues when two grid functions have different dimensions but should be updated by the same kernel. Recall the packing of `u`, `v` in the velocity update for the acoustic wave equation. In 2D `u` would have an extra grid point in the x-direction compared to `v`, whereas the opposite would be true for the y-direction. That is, `v` would have an extra grid point compared to `u` in this case.

Example of three-point zero-fill.
$$\begin{align}
Z = 
\begin{bmatrix}
-1 & 1  \\
0 & 0 \\
 0 & 0 \\
 0 & 0 
\end{bmatrix}
\end{align}
$$

The function call to accomplish this could simply be achieved using numpy slicing, or 

In [ ]:
zeros_left = fills.zeros(num_repeats=3,  block=np.array([-1.0, 1.0]), reverse=False)

## Updating operators

Naturally, we would like to avoid to manually rebuild our operators whenever some form of fill-in is necessary. In this case, the `fill` class becomes useful. This class rebuilds an operator by telling it what type of fill to perform, where to perform the fill, and any additional arguments need for the specific type of fill.

In [ ]:
# Modify the old derivative operator by making the boundary size 4 points wide by repeating the interior stencil
filler = fills.Fill(D, i_left, i_int, 4)
Dleft = filler.banded()

The last piece of information that we need is the updated bounds. Perhaps these could be obtained from `Fills` via indices:

In [ ]:
i_left, i_int = filler.indices()

In the above, the indices `i_left` and `i_int` would look the same as before (same label), but contain the updated bounds (assuming that it is a good idea to put bounds information on an index, which I think it is). 

If the operator also has a right boundary (it probably does), then we probably want to update it to. We can accomplish this by feeding the newly created operator into `Fills` again. 

In [ ]:
filler = fills.Fill(D, i_left, i_int, 4)
Dleft = filler.banded()
i_left, i_int = filler.indices()
filler = fills.Fill(Dleft, i_right, i_int, 4)
Dnew = filler.banded()
i_right, i_int = filler.indices()

